# Step 5: Find matches within a search radius

In [11]:
from functools import partial
import numpy as np
import os
from phathom.utils import pickle_load
from phathom.registration.coarse import rigid_transformation
from phathom.registration.pcloud import radius_matching, rotation_matrix

In [5]:
working_dir = '/home/jswaney/coregistration/'

In [6]:
# Load blobs
fixed_blobs_path = 'fixed/blobs.npy'
moving_blobs_path = 'moving/blobs.npy'

fixed_blobs = np.load(os.path.join(working_dir, 
                                   fixed_blobs_path))
moving_blobs = np.load(os.path.join(working_dir, 
                                    moving_blobs_path))
print('keypoints:', fixed_blobs.shape, moving_blobs.shape)

keypoints: (23637704, 3) (21672500, 3)


In [7]:
# Load features
fixed_features_path = 'fixed/features.npy'
moving_features_path = 'moving/features.npy'

fixed_feat = np.load(os.path.join(working_dir, 
                                  fixed_features_path))
moving_feat = np.load(os.path.join(working_dir, 
                                   moving_features_path))
print('features:', fixed_feat.shape, moving_feat.shape)

features: (23637704, 6) (21672500, 6)


In [8]:
# Load the rigid transformation
transformation_path = 'rigid_transformation.pkl'

transformation_dict = pickle_load(os.path.join(working_dir, 
                                               transformation_path))
t = transformation_dict['t']
center = transformation_dict['center']
theta = transformation_dict['theta']
print('Loaded rigid transformation for original image')
print('Rotation (deg):', theta*180/np.pi)
print('Center (px):', center)
print('Translation (px):', t)

Loaded rigid transformation for original image
Rotation (deg): [-160.86788649 -180.         -180.        ]
Center (px): [1989.03848673 5334.33099033 3145.78665711]
Translation (px): [   2.90338001 -370.8526147  -191.62759004]


In [15]:
# Roghly align fixed_pts
r = rotation_matrix(theta)
transformation = partial(rigid_transformation, t=t, r=r, center=center)
transformed_pts = transformation(pts=fixed_32cmstblobs)

In [17]:
# Match to moving_pts within search radius
radius = 180
nb_workers = None
batch_size = None
matching_kwargs = {'max_fdist': 2,
                   'prom_thresh': 0.3}

idx_fixed, idx_moving = radius_matching(transformed_pts, 
                                        moving_blobs, 
                                        fixed_feat, 
                                        moving_feat, 
                                        radius, 
                                        nb_workers, 
                                        batch_size, 
                                        matching_kwargs)
print('Found {} matches'.format(idx_fixed.shape[0]))

Found 112742 matches


In [18]:
# Save matching indices
fixed_idx_path = 'fixed/match_idx.npy'
moving_idx_path = 'moving/match_idx.npy'

np.save(os.path.join(working_dir, fixed_idx_path), idx_fixed)
np.save(os.path.join(working_dir, moving_idx_path), idx_moving)